In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
from torchsummary import summary

In [3]:
import numpy as np

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [5]:
## computes output shape for both convolutions and pooling layers
def output_shape(in_dim,stride,padding,kernel,dilation=1):
    out_dim = np.floor((in_dim + 2*padding - dilation*(kernel-1)-1)/stride+1).astype(int)
    return out_dim

In [6]:
class Encoder(nn.Module):
    def __init__(self, n_layers, out_channels, kernel_sizes, pooling_layers, paddings, strides, dim, activations, input_c, input_dim, latent_dim):
        super(Encoder, self).__init__()
        if dim == '1D':
            self.conv = nn.Conv1d
            self.pool = nn.MaxPool1d
        elif dim == '2D':
            self.conv = nn.Conv2d
            self.pool = nn.MaxPool2d
        else:
            raise Exception("Invalid data dimensionality (must be either 1D or 2D).")
            
        self.model = nn.ModuleList()
        current_channels = input_c
        current_dim      = input_dim
        for ii in range(n_layers):
            conv = self.conv(current_channels, out_channels[ii], kernel_sizes[ii], strides[ii], paddings[ii])
            self.model.append(conv)
            current_channels =  out_channels[ii]
            current_dim      =  output_shape(current_dim, strides[ii], paddings[ii],kernel_sizes[ii])
            
            if activations[ii]!='None':
                gate = getattr(nn, activations[ii])()
                self.model.append(gate)
                
            if pooling_layers[ii][0]:
                # kernel_size, stride, padding
                pool = self.pool(pooling_layers[ii][1], pooling_layers[ii][2], pooling_layers[ii][3])
                self.model.append(pool)
                current_dim = output_shape(current_dim, pooling_layers[ii][2],pooling_layers[ii][3],pooling_layers[ii][1])
                
        self.model.append(nn.Flatten())
        if dim == '1D':
            current_shape = current_channels*current_dim
        else:
            current_shape = current_channels*current_dim**2  
        self.model.append(nn.Linear(current_shape,latent_dim))

            
    def forward(self, x):
        for i, l in enumerate(self.model):
            x = l(x)
        return x




In [ ]:
class Encoder(nn.Module):
    def __init__(self, n_layers, out_channels, kernel_sizes, pooling_layers, paddings, strides, dim, activations, input_c, input_dim, latent_dim):
        super(Encoder, self).__init__()
        if dim == '1D':
            self.conv = nn.Conv1d
            self.pool = nn.MaxPool1d
        elif dim == '2D':
            self.conv = nn.Conv2d
            self.pool = nn.MaxPool2d
        else:
            raise Exception("Invalid data dimensionality (must be either 1D or 2D).")
            
        self.model = nn.ModuleList()
        self.dims        = []
        self.channels    = []
        
        current_channels = input_c
        current_dim      = input_dim
        for ii in range(n_layers):
            conv = self.conv(current_channels, out_channels[ii], kernel_sizes[ii], strides[ii], paddings[ii])
            self.model.append(conv)
            current_channels =  out_channels[ii]
            current_dim      =  output_shape(current_dim, strides[ii], paddings[ii],kernel_sizes[ii])
            
            if activations[ii]!='None':
                gate = getattr(nn, activations[ii])()
                self.model.append(gate)
                
            if pooling_layers[ii][0]:
                # kernel_size, stride, padding
                pool = self.pool(pooling_layers[ii][1], pooling_layers[ii][2], pooling_layers[ii][3])
                self.model.append(pool)
                current_dim = output_shape(current_dim, pooling_layers[ii][2],pooling_layers[ii][3],pooling_layers[ii][1])
                
        self.model.append(nn.Flatten())
        if dim == '1D':
            current_shape = current_channels*current_dim
        else:
            current_shape = current_channels*current_dim**2  
        self.model.append(nn.Linear(current_shape,latent_dim))

            
    def forward(self, x):
        for i, l in enumerate(self.model):
            x = l(x)
        return x

In [ ]:
class Decoder(nn.Module):
    def __init__(self, n_layers, out_channels, kernel_sizes, pooling_layers, paddings, strides, dim, activations, input_c, input_dim, latent_dim):
        super(Encoder, self).__init__()
        if dim == '1D':
            self.conv = nn.Conv1d
            self.pool = nn.MaxPool1d
        elif dim == '2D':
            self.conv = nn.Conv2d
            self.pool = nn.MaxPool2d
        else:
            raise Exception("Invalid data dimensionality (must be either 1D or 2D).")
            
        self.model = nn.ModuleList()
        current_channels = input_c
        current_dim      = latent_dim
        
        self.model.append(nn.Linear(latent_dim))
        
        for ii in range(n_layers):
            conv = self.conv(current_channels, out_channels[ii], kernel_sizes[ii], strides[ii], paddings[ii])
            self.model.append(conv)
            current_channels = out_channels[ii]
            current_dim      = output_shape(current_dim, strides[ii], paddings[ii],kernel_sizes[ii])
            
            if activations[ii]!='None':
                gate = getattr(nn, activations[ii])()
                self.model.append(gate)
                
            if pooling_layers[ii][0]:
                # kernel_size, stride, padding
                pool = self.pool(pooling_layers[ii][1], pooling_layers[ii][2], pooling_layers[ii][3])
                self.model.append(pool)
                current_dim = output_shape(current_dim, pooling_layers[ii][2],pooling_layers[ii][3],pooling_layers[ii][1])
                
        self.model.append(nn.Flatten())
        if dim == '1D':
            current_shape = current_channels*current_dim
        else:
            current_shape = current_channels*current_dim**2  
        

            
    def forward(self, x):
        for i, l in enumerate(self.model):
            x = l(x)
        return x

In [ ]:
# class Autoencoder(nn.Module):
#     def __init__(self, n_layers, in_channels, out_channels, kernel_sizes, pooling_layers, paddings, strides):
#         super(Autoencoder, self).__init__()
#         self.encoder = Encoder(n_layers, in_channels, out_channels, kernel_sizes, pooling_layers, paddings, strides)
#         self.decoder = Decoder(n_layers, in_channels, out_channels, kernel_sizes, pooling_layers, paddings, strides)

#     def forward(self, x):
#         x = self.encoder(x)
#         x = self.decoder(x)
#         return x

In [7]:
n_layers     = 2
out_channels = [2,4]
kernel_sizes = [2,4]
pooling_layers = [[False, None, None, None], [True, 2, 1, 0]]
paddings     = [0,0]
strides      = [2,4]
dim          = '1D'
activations  = ['ReLU', 'ReLU']
latent_dim   = 4
input_c      = 1 
input_dim    = 100

In [8]:
AE = Encoder(n_layers, out_channels, kernel_sizes, pooling_layers, paddings, strides, dim, activations, input_c, input_dim, latent_dim)

In [9]:
AE.to(device)

<generator object Module.parameters at 0x2aaba7bba6d0>

In [10]:
data =  torch.randn(1,1,100).to('cuda')

In [11]:


AE.forward(data)

tensor([[ 0.0868,  0.3251, -0.1661, -0.2457]], device='cuda:0',
       grad_fn=<AddmmBackward>)

In [14]:
summary(AE, data.shape[1::])


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1                [-1, 2, 50]               6
              ReLU-2                [-1, 2, 50]               0
            Conv1d-3                [-1, 4, 12]              36
              ReLU-4                [-1, 4, 12]               0
         MaxPool1d-5                [-1, 4, 11]               0
           Flatten-6                   [-1, 44]               0
            Linear-7                    [-1, 4]             180
Total params: 222
Trainable params: 222
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [14]:
AE.device()

AttributeError: 'Encoder' object has no attribute 'device'